# Getting viral sequences from metagenomic sequencing data
Anneliek ter Horst, May 2022

We are starting with metagenomic sequencing data from Bodega Bay, CA. Part of this is done using Snakemake

1. Clean sequencing reads: remove adapter sequences using trimmomatic
2. Clean sequencing reads: remove PhiX spike using bbduk
3. Assembly into contigs using MEGAHIT

## Trimmomatic
- http://www.usadellab.org/cms/?page=trimmomatic
- Bolger, A. M., Lohse, M., & Usadel, B. (2014). Trimmomatic: A flexible trimmer for Illumina Sequence Data. Bioinformatics, btu170.

In [ ]:
# trimmomatic snakefile
# run trimmomatic on raw reads
rule trimmomatic:
    input:
        forward_raw_1 = "reads/raw/{sample}_L001_R1_001.fastq.gz",
        reverse_raw_1 = "reads/raw/{sample}_L001_R2_001.fastq.gz",
    output:
        forward_paired_1 = "reads/trimmomatic/{sample}_L001_R1_paired.fq.gz",
        forward_unpaired_1 = "reads/trimmomatic/unpaired/{sample}_L001_R1_unpaired.fq.gz",
        reverse_paired_1 = "reads/trimmomatic/{sample}_L001_R2_paired.fq.gz",
        reverse_unpaired_1 = "reads/trimmomatic/unpaired/{sample}_L001_R2_unpaired.fq.gz",
        check = "reads/trimmomatic/out/{sample}_trim_done.txt"
        
    message: "Trimming Illumina adapters from {input.forward_raw_1} and {input.reverse_raw_1}"
    shell:'''
        module load java
        java -jar /programs/Trimmomatic-0.39/trimmomatic-0.39.jar PE -threads 8 -phred33 \
        {input.forward_raw_3} {input.reverse_raw_3} \
        {output.forward_paired_3} {output.forward_unpaired_3} \
        {output.reverse_paired_3} {output.reverse_unpaired_3} \
        ILLUMINACLIP:/programs/Trimmomatic-0.39/adapters/TruSeq3-PE.fa:2:30:10 \
        SLIDINGWINDOW:4:30 MINLEN:50 && touch {output.check}
        '''
        

# Removing phiX spike using bbmap
- https://jgi.doe.gov/data-and-tools/software-tools/bbtools/
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0185056

In [ ]:
# remove phiX spike from trimmed reads that are paired and unpaired
rule remove_phix:
    input:
        check = "reads/trimmomatic/out/{sample}_trim_done.txt",
        forward_trim = "reads/trimmomatic/{sample}_R1_paired.fq.gz",
        reverse_trim = "reads/trimmomatic/{sample}_R2_paired.fq.gz",
        unpaired_1 = "reads/trimmomatic/unpaired/{sample}_R1_unpaired.fq.gz",
        unpaired_2 = "reads/trimmomatic/unpaired/{sample}_R2_unpaired.fq.gz",
        reference = "/programs/bbduk/phix174_ill.ref.fa"
    output:
        forward_rm = "reads/rmphix/{sample}_R1_rmphix.fq.gz",
        reverse_rm = "reads/rmphix/{sample}_R2_rmphix.fq.gz",
        unpaired_1 = "reads/rmphix/unpaired/{sample}_R1_unpaired.fq.gz",
        unpaired_2 = "reads/rmphix/unpaired/{sample}_R2_unpaired.fq.gz",
        out_stats_un_1 = "reads/rmphix/unpaired/stats/{sample}_R1_stats.txt",
        out_stats_un_2 = "reads/rmphix/unpaired/stats/{sample}_R2_stats.txt",
        out_stats = "reads/rmphix/stats/{sample}_stats.txt",
        check = "reads/rmphix/out/{sample}_rmphix_done.txt"

    # print a message on what its doing
    message: "removing PhiX spike and host reads from {input.forward_trim} and {input.reverse_trim}"
    shell:'''
    module load java
    module load bbmap

    bbduk.sh in1={input.forward_trim} in2={input.reverse_trim} threads=8 \
    out1={output.forward_rm} out2={output.reverse_rm} stats={output.out_stats} \
    ref={input.reference} k=31 \
    hdist=1 && bbduk.sh in={input.unpaired_1} out={output.unpaired_1} threads=8 \
    stats={output.out_stats_un_1} ref={input.reference} k=31 \
    hdist=1 && bbduk.sh in={input.unpaired_2} out={output.unpaired_2} threads=8 \
    stats={output.out_stats_un_2} ref={input.reference} k=31 \
    hdist=1 && touch {output.check}
    '''


# Assembly of the clean reads using MEGAHIT
- https://github.com/voutcn/megahit
- https://academic.oup.com/bioinformatics/article/31/10/1674/177884

In [ ]:
# megahit needs a temp folder, as snakemake doesn't take folders as an output (thanks Taylor)

rule PE_assembly:
    input: 
        forward_read = "reads/trimmomatic/{sample}_L002_R1_paired.fq.gz", 
        reverse_read = "reads/trimmomatic/{sample}_L002_R2_paired.fq.gz",
        unpaired_1 = "reads/trimmomatic/unpaired/{sample}_L002_R1_unpaired.fq.gz",
        unpaired_2 = "reads/trimmomatic/unpaired/{sample}_L003_R1_unpaired.fq.gz", 
    output:
        check = "assemblies/out/{sample}_assem_done.txt",
        out_contig = "assemblies/megahit_final/{sample}.contigs.fa" 
    params:
        output_folder = "assemblies/megahit_final",
        output_temp = "assemblies/megahit_temp"
    message: "paired end assembly on {input.forward_read}"
    shell:'''
    mkdir -p  assemblies/megahit_temp/
    module load megahit
    
    # megahit does not allow force overwrite, so each assembly needs to take place in it's own directory. 
    megahit -1 {input.forward_read} -2 {input.reverse_read} \
    -r {input.unpaired_1},{input.unpaired_2} \
    -t 16 --continue --k-min 27 --min-contig-len 1000 --presets meta-large -m 0.095 \
    --out-dir {params.output_temp}/{wildcards.sample} \
    --out-prefix {wildcards.sample} && \
    mv {params.output_temp}/{wildcards.sample}/{wildcards.sample}.contigs.fa \
    {params.output_folder} && touch {output.check}
    '''
        

## Renaming contigs and split on lenght
- MEGAHIT outputs contigs with spaces in filenames, don't like that
- Only keep contigs > 10kb for predicting viruses
- Using bbmap package
- https://jgi.doe.gov/data-and-tools/software-tools/bbtools/
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0185056

In [ ]:
# rename contigs and  split off contigs >10kb
rule rename_contigs:
    input:
        contigs = "assemblies/megahit_final/{sample}.contigs.fa",
        check = "assemblies/out/{sample}_assem_done.txt"
    output:
        contigs_renamed = "outputs/megahit_final/{sample}.contigs_renamed.fa",
        assembly_stats = "outputs/megahit_final/stats_assembly/{sample}.megahit.stats.txt",
        contigs_10kb = "outputs/megahit_final/{sample}.contigs_10kb.fa",
        check = "assemblies/out/{sample}_reformat_done.txt"
    # print a message on what its doing
    message: "renaming contigs of sample {input.contigs}"
    shell:'''
    module load bbmap
    rename.sh in={input.contigs} out={output.contigs_renamed} prefix={wildcards.sample} && \
    reformat.sh in={output.contigs_renamed} out={output.contigs_10kb} minlength=10000 &&
    stats.sh in={output.contigs_renamed} gc={output.assembly_stats} gcformat=3 && touch {output.check}
    '''

## Use VIBRANT to predict viral contigs from all contigs
- https://github.com/AnantharamanLab/VIBRANT
- https://pubmed.ncbi.nlm.nih.gov/32522236/

In [ ]:
# Same thing with a temp folder for snakemake as with MEGAHIT
# run vibrant
rule vibrant:
    input:
        contigs = "outputs/megahit_final/{sample}.contigs_10kb.fa",
        check = "assemblies/out/{sample}_reformat_done.txt"
    output:
        vibrant_contig = "outputs/vibrant_final/{sample}.contigs_10kb.phages_combined.fna",
        check = "outputs/out/{sample}_vibrant_done.txt"
    params:
        output_folder = "outputs/vibrant_final",
        output_temp = "outputs/vibrant_temp"
    # print a message on what its doing
    message: "running vibrant on {input.contigs}"
    shell:'''
    mkdir -p  outputs/vibrant_temp/
    
    set +u
    source ~/.bashrc
    conda activate vibrant
    set -u
    
    VIBRANT_run.py -i {input.contigs} -folder {params.output_temp}/{wildcards.sample} \
    -t 8 -l 10000 -o 4 -virome && \
    mv {params.output_temp}/{wildcards.sample}/VIBRANT_{wildcards.sample}.contigs_10kb/VIBRANT_phages_{wildcards.sample}.contigs_10kb/{wildcards.sample}.contigs_10kb.phages_combined.fna \
    {params.output_folder} && touch {output.check} 
    '''

## Use dRep to dereplicate viral contigs. 
- https://drep.readthedocs.io/en/latest/
- https://www.nature.com/ismej/articles

In [ ]:
# run drep on all viral contigs predicted by vibrant (n=17703) + all found by PIGEON (581)


# split contigs into individual fastas (dRep accepts individual contigs only)
mkdir contigs
cd ./contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' ../*.fa 


# Activate dRep conda instance
source ~/.bashrc
conda activate drep

# Load other programs that dRep needs
module load mummer
module load mash
module load bowtie2

# Run dRep at 95% ANI over 85% of length of longest contigs
dRep dereplicate ./drep --S_algorithm ANImf --ignoreGenomeQuality -l 10000 -sa 0.95 -nc 0.85 \
-g ./contigs/*.fa -p 14


## Use Bowtie2 to map back reads to viral contigs
- Make a bowtie2 index
- map reads to index using bowtie2
- http://bowtie-bio.sourceforge.net/bowtie2/index.shtml
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3322381/
- Convert samfiles to bamfiles using samtools
- index bamfiles using samtools
- http://www.htslib.org/
- https://academic.oup.com/bioinformatics/article/25/16/2078/204688


In [ ]:
# First build bowtie2 index from viral contigs (n=14512, dereplicated)

# concat all dereplicated contigs
cat ./drep/dereplicated_genomes/*.fa >> ./220315_wetland_votus_drep_animf.fa

# build index file for mapping
module load bowtie2
bowtie2-build 220315_wetland_votus_drep_animf.fa ./bowtie2/220315_wetland_votus_drep -p 10

In [ ]:
# Now map the reads to this bowtie2 index
# Then sam -> bam
# Then index bamfile
# Run bowtie
rule bowtie:
    input:
        index = 'outputs/bowtie2/220315_wetland_votus_drep',
        fw_read_1 = "reads/trimmomatic/{sample}_L002_R1_paired.fq.gz", 
        rv_read_1 = "reads/trimmomatic/{sample}_L002_R2_paired.fq.gz",
    output:
        samfile = "outputs/mapping/{sample}.sam",
        bamfile = "outputs/mapping/{sample}.bam",
        check = "outputs/mapping/out/{sample}_done.txt"

        
    # print a message on what its doing
    message: "running bowtie on {input.fw_read_1}"
    shell:'''
    module load bowtie2
    module load samtools
    
    bowtie2 --threads 12 -x ./outputs/bowtie2/220315_wetland_votus_drep \
    -1 {input.fw_read_1},{input.fw_read_2}\
    -2 {input.rv_read_1},{input.rv_read_2} \
    -S {output.samfile} --no-unal --sensitive && \
    samtools view -@ 12 -F 4 -bS {output.samfile} | samtools sort > {output.bamfile} && \
    samtools index {output.bamfile} && touch {output.check}
    '''

## Use CoverM to make a coverage table
- https://github.com/wwood/CoverM
- https://wwood.github.io/CoverM/coverm-genome.html#author

In [ ]:
# Activate coverM instance in conda
source ~/.bashrc
conda activate coverm

# make a coverage table, where the min amount of the contig that has to be covered is 75%
coverm contig -m mean --min-covered-fraction 0.75 -b *.bam > 211220_coverM_wetland_mean_75.tsv


## Use Sortmerna to extract 16S reads from the metagenomes
- https://bioinfo.lifl.fr/RNA/sortmerna/
- https://academic.oup.com/bioinformatics/article/28/24/3211/246053

In [ ]:
# Load other programs that sortmerna needs
module load bbmap
module load samtools

# activate sortmerna environment
source ~/.bashrc
conda activate sortmerna

# do this for each fastq file (paired reads)
for f in *_R1*.gz
do 
sortmerna --ref ./16S_db/silva-bac-16s-database-id85.fasta \
--ref ./16S_db/silva-arc-16s-database-id95.fasta \
--reads $1 \
--reads ${1%R1*}R2.fastq.gz \
--workdir ./sortmerna/${1%_R1*} \
--fastx \
--aligned ./sortmerna/sort_results/${1%R1*}rrna \
-v --threads 24 --paired_out
done

## Use RDP tools to classify the found 16S rRNA reads
- http://rdp.cme.msu.edu/
- https://academic.oup.com/nar/article/42/D1/D633/1063201?login=true

In [ ]:
# Activate conda env
source ~/.bashrc
conda activate rdptools

# Use the classifier command
for f in *.fq.gz 
do 
echo $f
java -jar ./miniconda3/envs/rdptools/bin/classifier.jar classify \
-g 16srrna \
-f fixrank \
-b ${f%%.fq.gz*}.bootstrap \
-h ${f%%.fq.gz*}.hier.tsv \
-o ${f%%.fq.gz*}.class.tsv \
./$f 
done


## Use CRASS to predict crispr - spacer regions in both host and virus 
- https://ctskennerton.github.io/crass/
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3664793/

In [ ]:
# activate crass environment
source ~/.bashrc
conda activate crass

# Do for each paired fastq file
for f in *R1*.gz 
do
crass $f ${f%%_R1*}_R2.fastq.gz -o ./crass/${f%%_R1*} -l 4
done

# Also look for spacers in the 2019 reads

# extract spacer and repeat sequences
for f in */crass.crispr; do
echo $f
crisprtools extract -o ${f%%crass.crispr*}spacers -s $f -x 
done


# all the repeats and spacers need a prefix with the sample name, cause there is multiple spacers/repeats with the same name
module load bbmap
for f in *
do
echo $f
cd $f 
# concatenate 
cat ./spacers/*.fa > all_spacers_crass.fa
# rename
rename.sh in=all_spacers_crass.fa out=all_spacers_crass_rename.fa prefix=$f addprefix=t
cd ..
done


# concatenate all spacers
cat */all_spacers_crass_rename.fa > all_spacers_crass.fa



In [ ]:
# Now use blast to match to the spacers:
# First make a fasta file of all 2019 and 2021 sequences

cat 220908_2019_BB_vOTUs.fa 220908_all_2019_2021_BB_votus.fa >> 220908_all_2019_2021_BB_votus.fa

# remove the duplicates, aka the ones that were in both files
source ~/.bashrc
conda activate seqkit

seqkit rmdup --quiet -n 220908_all_2019_2021_BB_votus.fa > 220908_all_2019_2021_BB_votus.fa.dedupe

In [ ]:

# Make a blastdb of all vOTU sequences (including all 2019 BB vOTUs.)
makeblastdb -in 220908_all_2019_2021_BB_votus.fa.dedupe -dbtype nucl -out vOTU_BB_db

# blast back blastn -eval 1e-10 and 100% nucl ident

# WE BLAST THE SPACER SEQUENCES TO THE VOTUS
blastn -task blastn-short \
-query all_spacers_crass.fa \
-db ../outputs/vOTU_BB_db -evalue 1e-10 -perc_identity 95 -outfmt 6 \
-out spacers_vOTUs_crass

In [ ]:
# extract spacer and repeat sequences
for f in BB_*/crass.crispr; do
echo $f
crisprtools extract -o ${f%%crass.crispr*}spacers -s $f -x 
done


# all the repeats and spacers need a prefix with the sample name, cause there is multiple spacers/repeats with the same name
module load bbmap
for f in *
do
echo $f
cd $f 
# concatenate 
cat ./spacers/*.fa > all_spacers_crass.fa
# rename
rename.sh in=all_spacers_crass.fa out=all_spacers_crass_rename.fa prefix=$f addprefix=t
cd ..
done


# concatenate all spacers
cat */all_spacers_crass_rename.fa > all_spacers_crass.fa

# dediplicate by sequeonce



In [ ]:
#!/bin/bash
#SBATCH -D /home/csantosm/wetup/scripts/
#SBATCH --job-name=iphop_split
#SBATCH --nodes=1
#SBATCH -t 1:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate IPHOP

set=${1}

full_db=/home/amhorst/2022_wetlands/outputs/220518_drep_all_vOTUs_recov_BB.fa
split_db=/home/amhorst/2022_wetlands/outputs/iphop/split_db

iphop split --input_file ${full_db} --split_dir ${split_db}

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

In [ ]:
# Run iphop for host predictions
## The db needs to be chmodded 

for f in *.fna
do
echo $f
sbatch iphop.sh $f
done

#!/bin/bash
#SBATCH --job-name=iphop_predict
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --mem=128GB
#SBATCH --ntasks=24
#SBATCH --partition=bmh

# for calculating the amount of time the job takes

source /home/csantosm/initconda
conda activate IPHOP

iphop predict \
--fa_file $1 \
--out_dir ../results/${1%.fna*} \
--db_dir /home/csantosm/databases/IPHOP_db/Sept_2021_pub \
--num_threads 24


In [ ]:
## Try vamb for binning 

In [ ]:
#!/bin/bash
#SBATCH --job-name=vamb
#SBATCH --nodes=1
#SBATCH -t 168:00:00
#SBATCH --ntasks=24
#SBATCH --partition=bmm


# loading modules
source /home/csantosm/initconda
conda activate VAMB

vamb --outdir ./vamb --fasta ./assemblies/coverage/all_bulk_contigs_renamed_cluster.fa \
--jgi /home/amhorst/2022_wetlands/assemblies/coverage/mapping/metabat_depth.txt --minfasta 50000

